# 휴먼지능정보공학과 201910803 박채희 기말시험

In [30]:
import pyspark

In [31]:
myConf=pyspark.SparkConf()
spark=pyspark.sql.SparkSession.builder.master("local").appName("myApp").config(conf=myConf).getOrCreate()

## 문제1: 서울시 지하철 호선별 역별 유/무임 승하차 인원 분석



'서울시 지하철 호선별 역별 유/무임 승하차 인원 정보'를 csv파일로 내려받아서 분석하세요

(https://data.seoul.go.kr/dataList/OA-12251/S/1/datasetView.do?tab=S)


## (1-1) '서울시 지하철 호선별 역별 유_무임 승하차 인원 정보.csv'를 데이터프레임으로 읽고, 스키마를 출력

(한글이 깨질 경우에는 엑셀에서 UTF로 저장하거나, 직접 csv파일을 수정하거나 우회적인 방법을 통해서 해결, 약간의 감점).

In [410]:
subway = spark\
            .read.option("charset", "euc-kr")\
            .option("header", "true")\
            .csv("서울시 지하철 호선별 역별 유_무임 승하차 인원 정보.csv")

## 문제 1-1의 답
한글이 깨지지않고 정상적으로 출력된다.

In [411]:
subway.printSchema()

root
 |-- 사용월: string (nullable = true)
 |-- 호선명: string (nullable = true)
 |-- 지하철역: string (nullable = true)
 |-- 유임승차인원: string (nullable = true)
 |-- 무임승차인원: string (nullable = true)
 |-- 유임하차인원: string (nullable = true)
 |-- 무임하차인원: string (nullable = true)
 |-- 작업일자: string (nullable = true)



## (1-2) 건수를 출력
## 문제 1-2의 답

In [412]:
subway.count()

55171

## (1-3) '호선명'별 '유임승차인원'의 순위를 매기고, 그 결과를 30줄 출력 (마지막 'rank' 컬럼)

```

+------+------+--------+------------+------------+------------+------------+--------+----+

|사용월|호선명|지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rank|

+------+------+--------+------------+------------+------------+------------+--------+----+

|201605|일산선|    화정|      515925|      139163|      535460|      142410|20160608|   1|

|201512|일산선|    화정|      508044|      136734|      529106|      139931|20160108|   2|

|201603|일산선|    화정|      502144|      140596|      526030|      144088|20160408|   3|

```



In [413]:
from pyspark.sql.types import FloatType

subway = subway.withColumn('유임승차인원_f', subway['유임승차인원'].cast(FloatType()))

In [414]:
from pyspark.sql.window import Window

win = Window.partitionBy("호선명").orderBy(F.col("유임승차인원_f").desc())

## 문제 1-3의 답

In [415]:
from pyspark.sql.functions import row_number

subway_rank=subway.withColumn("rank", row_number().over(win))
subway_rank.select('사용월','호선명','지하철역','유임승차인원','무임승차인원','유임하차인원','무임하차인원','작업일자','rank').show(30)

+------+------+--------+------------+------------+------------+------------+--------+----+
|사용월|호선명|지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rank|
+------+------+--------+------------+------------+------------+------------+--------+----+
|201605|일산선|    화정|      515925|      139163|      535460|      142410|20160608|   1|
|201512|일산선|    화정|      508044|      136734|      529106|      139931|20160108|   2|
|201603|일산선|    화정|      502144|      140596|      526030|      144088|20160408|   3|
|201703|일산선|    화정|      499911|      141327|      529944|      145502|20170403|   4|
|201510|일산선|    화정|      499388|      520595|      140545|      144148|20151108|   5|
|201905|일산선|    화정|      499218|      149790|      520966|      152915|20190603|   6|
|201612|일산선|    화정|      497514|      132807|      521734|      136086|20170108|   7|
|201805|일산선|    화정|      494994|      139239|      515748|      142997|20180603|   8|
|201610|일산선|    화정|      492146|      137462|      513849|      140303|201611

## (1-4) '호선명'별 '유임승차인원'이 순위가 가장 높은 경우만 30건 출력 (마지막 'rank' 컬럼이 모두 1)

```

+------+--------------+--------------------+------------+------------+------------+------------+--------+----+

|사용월|        호선명|            지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rank|

+------+--------------+--------------------+------------+------------+------------+------------+--------+----+

|201605|        일산선|                화정|      515925|      139163|      535460|      142410|20160608|   1|

|201905|        장항선|                아산|      109243|       12901|       93737|       11808|20190603|   1|

|201512|        경부선|              영등포|     1439049|      280623|     1545631|      280969|20160108|   1|

```

## 문제 1-4의 답


In [416]:
subway_rank.select('사용월','호선명','지하철역','유임승차인원','무임승차인원','유임하차인원','무임하차인원','작업일자','rank')\
           .filter(subway_rank.rank==1).show(30)

+------+--------------+--------------------+------------+------------+------------+------------+--------+----+
|사용월|        호선명|            지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rank|
+------+--------------+--------------------+------------+------------+------------+------------+--------+----+
|201605|        일산선|                화정|      515925|      139163|      535460|      142410|20160608|   1|
|201905|        장항선|                아산|      109243|       12901|       93737|       11808|20190603|   1|
|201512|        경부선|              영등포|     1439049|      280623|     1545631|      280969|20160108|   1|
|201905|    우이신설선|        북한산보국문|      161867|       45041|      141742|       45814|20190603|   1|
|201512|        분당선|                야탑|      769442|      157846|      809197|      156034|20160108|   1|
|201512|         7호선|      가산디지털단지|     1245731|       82588|     1243464|       78424|20160108|   1|
|201905|        수인선|              인하대|      200474|       28246|      189885|   

## (1-5) '호선명'별 '유임승차인원'의 zscore를 계산하고 출력. 윈도우 함수를 사용하여야 한다.

```

+------+------------+------------------+------------------+--------------------+

|호선명|유임승차인원|        meanByLine|          sdByLine|              zscore|

+------+------------+------------------+------------------+--------------------+

|일산선|      237714|231630.21716649432|127730.48519577553| 0.04762984203951724|

|일산선|      363144|231630.21716649432|127730.48519577553|  1.0296193788971475|

|일산선|      294566|231630.21716649432|127730.48519577553|  0.4927232738295993|

```

In [417]:
import sys

byAll = Window.rowsBetween(-sys.maxsize, sys.maxsize)

In [418]:
from pyspark.sql import functions as F
subway_rank = subway_rank.withColumn("meanByLine", F.avg(subway_rank['유임승차인원_f']).over(byAll))

In [419]:
subway_rank = subway_rank.withColumn("sdByLine", F.stddev(subway_rank['유임승차인원_f']).over(byAll))

In [420]:
subway_rank = subway_rank.withColumn("zscore", (F.col('유임승차인원_f')-F.col('meanByLine'))/F.col('sdByLine'))

## 문제 1-5의 답

In [421]:
subway_rank.select('호선명','유임승차인원','meanByLine','sdByLine','zscore').show(30)

+------+------------+-----------------+-----------------+--------------------+
|호선명|유임승차인원|       meanByLine|         sdByLine|              zscore|
+------+------------+-----------------+-----------------+--------------------+
| 1호선|      516353|302546.6513023146|322895.6016002317|  0.6621531778013912|
| 1호선|      119396|302546.6513023146|322895.6016002317| -0.5672132119317885|
| 1호선|      271006|302546.6513023146|322895.6016002317|-0.09768064707602991|
| 1호선|      315789|302546.6513023146|322895.6016002317| 0.04101123902604408|
| 1호선|      797960|302546.6513023146|322895.6016002317|  1.5342833604498682|
| 1호선|      222630|302546.6513023146|322895.6016002317| -0.2474999687399187|
| 1호선|      210503|302546.6513023146|322895.6016002317|-0.28505699937118173|
| 1호선|      433793|302546.6513023146|322895.6016002317|  0.4064668210011047|
| 1호선|      362377|302546.6513023146|322895.6016002317| 0.18529316720690342|
| 1호선|     1086374|302546.6513023146|322895.6016002317|  2.4274946602342413|
| 

주어진 dataframe과 같은 모양을 만들기 위해 filter을 적용해 일산선만 출력하게 하였다.

In [425]:
subway_rank.select('호선명','유임승차인원','meanByLine','sdByLine','zscore').filter(subway_rank.호선명=='일산선').show(30)

+------+------------+-----------------+-----------------+--------------------+
|호선명|유임승차인원|       meanByLine|         sdByLine|              zscore|
+------+------------+-----------------+-----------------+--------------------+
|일산선|      237714|302546.6513023146|322895.6016002317|-0.20078517942335475|
|일산선|      363144|302546.6513023146|322895.6016002317| 0.18766854796835955|
|일산선|      294566|302546.6513023146|322895.6016002317|-0.02471588731083189|
|일산선|      208122|302546.6513023146|322895.6016002317|-0.29243089975322484|
|일산선|      237249|302546.6513023146|322895.6016002317|-0.20222527336608892|
|일산선|      159355|302546.6513023146|322895.6016002317|-0.44346113911949875|
|일산선|           7|302546.6513023146|322895.6016002317| -0.9369581059728423|
|일산선|      171040|302546.6513023146|322895.6016002317| -0.4072729719778884|
|일산선|      147292|302546.6513023146|322895.6016002317|  -0.480819963272622|
|일산선|      432339|302546.6513023146|322895.6016002317|  0.4019638175758671|
|일산선|       

# 문제2: 회귀분석



sklearn의 make_regression함수를 다음과 같이 사용하여 데이터를 생성하고, 문제를 푸세요.




In [61]:
from sklearn.datasets import make_regression

X, y, coef = make_regression(n_samples = 200,

                            n_features = 4,

                            n_informative = 3,

                            n_targets = 1,

                            noise = 0.0,

                            coef = True) 

## (2-1) gradient 방법으로 회귀식을 계산하고, 계수를 출력 .
 또한 [식-1]의 'coef'를 출력하고, gradient방법에서 도출된 계수와 일치하는지 비교하세요.
 
 
 주의: 반복이 많으면 감점 (예를 들어, 1000 이상이면 많다고 간주할 것임)

In [62]:
import numpy as np
alpha = 0.02
theta=np.array(np.ones([X.shape[1]]))

def gradientDescent(x,y,theta,alpha,numIterations):
    for i in range(numIterations):
        h=np.dot(x,theta)
        error=h-y
        cost=np.sum((h-y)**2)/len(x) 
        gradient=np.dot(x.T,error)*2/len(x) 
        theta-=alpha*gradient
        if i%100==0:
            print("Iteration {0} | theta{1} Cost {2:.5f}".format(i,theta,cost))

    return theta

## 문제 2-1의 답

In [63]:
w4,w3,w2,w1=gradientDescent(X, y, theta, alpha, 800) 

Iteration 0 | theta[2.189559   4.51014437 2.13122874 1.31561508] Cost 8451.28505
Iteration 100 | theta[28.32716217 80.36645767 21.49902273  0.69455855] Cost 1.86599
Iteration 200 | theta[2.87432920e+01 8.14412928e+01 2.12635793e+01 3.02323371e-02] Cost 0.00181
Iteration 300 | theta[2.87539145e+01 8.14661983e+01 2.12450039e+01 1.13845632e-03] Cost 0.00000
Iteration 400 | theta[2.87542608e+01 8.14670001e+01 2.12441954e+01 4.19883869e-05] Cost 0.00000
Iteration 500 | theta[2.87542730e+01 8.14670291e+01 2.12441635e+01 1.54977206e-06] Cost 0.00000
Iteration 600 | theta[2.87542734e+01 8.14670302e+01 2.12441623e+01 5.74005332e-08] Cost 0.00000
Iteration 700 | theta[2.87542734e+01 8.14670302e+01 2.12441623e+01 2.13196032e-09] Cost 0.00000


In [67]:
print("예측 결과: %f %f %f %f"%(w4,w3,w2,w1))
print("[식-1] coef: %f %f %f %f"%(coef[0],coef[1],coef[2],coef[3]))

예측 결과: 28.754273 81.467030 21.244162 0.000000
[식-1] coef: 28.754273 81.467030 21.244162 0.000000


## (2-2) 스파크 데이터프레임을 생성 (이후 계속 스파크로 푸세요)

In [68]:
import pandas as pd
pdDf=pd.DataFrame(X)
pdDf['y']=y

spark_Df=spark.createDataFrame(pdDf)

## 문제 2-2의 답

In [69]:
spark_Df.show(5)

+--------------------+--------------------+-------------------+-------------------+-------------------+
|                   0|                   1|                  2|                  3|                  y|
+--------------------+--------------------+-------------------+-------------------+-------------------+
|-0.44210614183925967|-0.10328126098717995|-0.8158114287377021|-1.6314768493361436| -38.45768887933029|
| 0.42024254610608586| -1.0356420650992118|-0.6098576185189238| 1.4694730959368787| -85.24282853349166|
| -1.5532948599582495| -1.7041092859727431|-1.5144964634770433|0.35864470881281507|-215.66679642330035|
| 0.18026375327910846| -0.8437234108294399| 0.3020888069736359| 0.6594824799790104| -57.13466369754052|
| -1.2291961099442235| -0.7817456854370127| 1.7978280065360968|0.40742271894443677|-60.837790469240545|
+--------------------+--------------------+-------------------+-------------------+-------------------+
only showing top 5 rows



## (2-3) 훈련, 테스트 데이터를 6:4 비율로 분리하고,

## 문제 2-3의 답

In [70]:
(trainDf, testDf) = spark_Df.randomSplit([0.6, 0.4])

## (2-4) 회귀모델링하고 계수와 절편 출력.

In [71]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(inputCols = ['0','1','2','3'], outputCol = 'features')

In [72]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='y', maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [73]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[va,lr])

In [74]:
model = pipeline.fit(trainDf)

In [75]:
modelTrainDf = model.transform(trainDf)

## 문제 2-4의 답

In [76]:
print("Coefficients: {}".format(model.stages[-1].coefficients))
print("Intercept: {:.3f}".format(model.stages[-1].intercept))

Coefficients: [28.5128058419206,81.19014343521607,20.978950746926113,0.0]
Intercept: 0.019


##실제와 예측을 10줄 출력.

In [77]:
modelTestDf=model.transform(testDf)

In [78]:
modelTestDf.select('y','prediction').show(10)

+-------------------+-------------------+
|                  y|         prediction|
+-------------------+-------------------+
| -56.33671458172286| -55.97915404524206|
|-159.13876490491322|  -158.040764129001|
|-100.01705255283309| -99.57824909180741|
|  85.95141246538168|  85.76391857765208|
|-124.54711438435373|-123.90258346851233|
|  21.75757529786695|   21.9106293994778|
| -35.51410545219552| -35.22705420563006|
|  60.67835721753735|  60.57454303695341|
|  32.05795416816564|   32.0048078501977|
|-128.29102376331213|-127.56485676162137|
+-------------------+-------------------+
only showing top 10 rows



## (2-5) 테스트 데이터에 대해 R2를 구해서 출력하세요.

In [79]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator=RegressionEvaluator(labelCol='y', predictionCol='prediction', metricName="r2")
f"r2: {evaluator.evaluate(modelTestDf):.8f}"

'r2: 0.99997392'

# 문제3: 텍스트 분류



다음은 영화리뷰이다.


## (3-1) 데이터프레임을 생성하세요.

* 영화리뷰를 파일로 작성하자. ipynb에서 직접 하거나 편리한 에디터를 사용함.

* 리뷰는 한 줄씩 저장하며, 각각의 긍정, 부정을 구분하여 레이블을 지정하고 함께 저장함.

* 파일로 작성하지 못하면, 이후 문제를 포기하지 말고 리스트로 만들거나 자신이 할 수 있는 방법을 통해서 만들어도 됨.


## 문제 3-1의 답
### space는 문장에 많이 섞여 있어 tab으로 문장과 label을 구별하는 review.txt를 만들어 파일로 저장하였다. 
### 부정은 0, 긍정은 1 label을 사용하였다.

In [430]:
review = spark\
    .read\
    .options(header='false', inferschema='true', delimiter='\t')\
    .csv('review.txt')

In [431]:
review.show(47)

+----------------------------------+---+
|                               _c0|_c1|
+----------------------------------+---+
|고구마 먹은 영화 유치해서 못보겠다|  0|
|  노잼이네요.. 왜 배우들이 출연...|  0|
|   제발 뻘짓 좀 그만하셨으면......|  0|
|         스토리가 매우 매우 아쉽다|  0|
|             영화 약간 이해가 안됨|  0|
|좋아하는 배우들인데 최근 영화라...|  0|
| 그지같네 도대체 이 따위 영화를...|  0|
|  스크린보다 시계를 더 많이 봄....|  0|
| 이때까지 본 영화중에 제일 최악...|  0|
|    나만 짜증.. 줄거리 한줄평.....|  0|
|로맨틱 코미디로 방향을 잡았으면...|  0|
| 똥 싸다가 막힌 느낌 안보는거를...|  0|
|  어우 핵노잼 영화가 왜 이렇게 ...|  0|
| 솔직히 재미없음 감독이 뭘 말할...|  0|
|   뭘 말하고싶은지 모르겠다 아쉽다|  0|
| 새벽에 케이블에서 볼거 없어서 ...|  0|
|         핵노잼 억지 착한척 오졌다|  0|
|오그라드는 대사는 기본에 옵션으...|  0|
| 역대급 호화캐스팅인데 스토리, ...|  0|
|  스토리 개 말도 안되는 억지 고...|  0|
| 어설픈 연출력 어설픈 결말 현실...|  0|
|   노잼 하 2시간 반 돌려내 이제...|  0|
| 재미없음 충분히 더 흥미롭게 다...|  0|
|  생각보다 더 감흥이 없어요. 액...|  0|
| 그냥 보지마세요 쓰레기 영화임 ...|  0|
|  지루해서 이거 다 보는데 두 달...|  0|
|        진짜 꼭 보세요 최고의 영화|  1|
|  엔드게임 이후 마블이 한 번 더...|  1|
| 정말 오랜만에 감동 울었네... 펑펑|  1|
|연출에서 등장인물 하나하나의 아...|  1|
|어렸을때는 그냥 액션과 화려함에..

In [432]:
review = review.withColumnRenamed("_c0", "sent")
review = review.withColumnRenamed("_c1", "label")

In [433]:
review.show(3)

+----------------------------------+-----+
|                              sent|label|
+----------------------------------+-----+
|고구마 먹은 영화 유치해서 못보겠다|    0|
|  노잼이네요.. 왜 배우들이 출연...|    0|
|   제발 뻘짓 좀 그만하셨으면......|    0|
+----------------------------------+-----+
only showing top 3 rows



## (3-2) 불용어 제거, 불필요 단어 정리

느낌표, 점, '...', 'ㅋㅋㅋ', 'ㅠ', 'ㄹㅇ' 불필요한 단어 제거

불용어 제거: '그', '수', '본', '등', '때', '걸', '다', '게', '날',

'짱', '거', '중', '더', '넘', '두', '달', '올', '뻔', '단', '개', '듯'

In [434]:
from pyspark.ml.feature import RegexTokenizer

re = RegexTokenizer(inputCol="sent", outputCol="wordsReg", pattern="\\s+")

In [435]:
reDf=re.transform(review)
reDf.show()

+----------------------------------+-----+--------------------------------+
|                              sent|label|                        wordsReg|
+----------------------------------+-----+--------------------------------+
|고구마 먹은 영화 유치해서 못보겠다|    0|  [고구마, 먹은, 영화, 유치해...|
|  노잼이네요.. 왜 배우들이 출연...|    0|  [노잼이네요.., 왜, 배우들이...|
|   제발 뻘짓 좀 그만하셨으면......|    0|  [제발, 뻘짓, 좀, 그만하셨으...|
|         스토리가 매우 매우 아쉽다|    0|  [스토리가, 매우, 매우, 아쉽다]|
|             영화 약간 이해가 안됨|    0|      [영화, 약간, 이해가, 안됨]|
|좋아하는 배우들인데 최근 영화라...|    0| [좋아하는, 배우들인데, 최근,...|
| 그지같네 도대체 이 따위 영화를...|    0|  [그지같네, 도대체, 이, 따위...|
|  스크린보다 시계를 더 많이 봄....|    0|  [스크린보다, 시계를, 더, 많...|
| 이때까지 본 영화중에 제일 최악...|    0|  [이때까지, 본, 영화중에, 제...|
|    나만 짜증.. 줄거리 한줄평.....|    0|    [나만, 짜증.., 줄거리, 한...|
|로맨틱 코미디로 방향을 잡았으면...|    0|  [로맨틱, 코미디로, 방향을, ...|
| 똥 싸다가 막힌 느낌 안보는거를...|    0|    [똥, 싸다가, 막힌, 느낌, ...|
|  어우 핵노잼 영화가 왜 이렇게 ...|    0|   [어우, 핵노잼, 영화가, 왜,...|
| 솔직히 재미없음 감독이 뭘 말할...|    0|  [솔직히, 재미없음, 감독이, ...|
|   뭘 말하고싶은지 모르겠

불필요한 단어 제거

In [436]:
def trim(wordList):
  cleaned=list()
  for w in wordList:
      cleaned.append(w.replace("!","").replace(".","").replace("...","").replace("ㅋㅋㅋ","").replace("ㅠ","").replace("ㄹㅇ",""))
  return cleaned

In [437]:
from pyspark.sql import functions as f
from pyspark.sql.types import ArrayType, StringType

trimUdf=f.udf(trim, ArrayType(StringType()))

In [438]:
review = tokDf.withColumn('words', trimUdf(f.col('tokens')))

In [439]:
review.show(3)

+----------------------------------+-----+------------------------------+------------------------------+
|                              sent|label|                        tokens|                         words|
+----------------------------------+-----+------------------------------+------------------------------+
|고구마 먹은 영화 유치해서 못보겠다|    0|[고구마, 먹은, 영화, 유치해...|[고구마, 먹은, 영화, 유치해...|
|  노잼이네요.. 왜 배우들이 출연...|    0|[노잼이네요.., 왜, 배우들이...|[노잼이네요, 왜, 배우들이, ...|
|   제발 뻘짓 좀 그만하셨으면......|    0|[제발, 뻘짓, 좀, 그만하셨으...|[제발, 뻘짓, 좀, 그만하셨으...|
+----------------------------------+-----+------------------------------+------------------------------+
only showing top 3 rows



불용어 생성

In [440]:
from pyspark.ml.feature import StopWordsRemover

stop = StopWordsRemover(inputCol="words", outputCol="nostops")
stop.setStopWords(['그', '수', '본', '등', '때', '걸', '다', '게', '날', '짱', '거', '중', '더', '넘', '두', '달', '올', '뻔', '단', '개', '듯'])

StopWordsRemover_4330104e6577

In [441]:
review=stop.transform(review)

## 문제 3-2의 답
### 정제된 단어를 nostops column으로 생성

In [442]:
review.show(10)

+----------------------------------+-----+-------------------------------+-------------------------------+-------------------------------+
|                              sent|label|                         tokens|                          words|                        nostops|
+----------------------------------+-----+-------------------------------+-------------------------------+-------------------------------+
|고구마 먹은 영화 유치해서 못보겠다|    0| [고구마, 먹은, 영화, 유치해...| [고구마, 먹은, 영화, 유치해...| [고구마, 먹은, 영화, 유치해...|
|  노잼이네요.. 왜 배우들이 출연...|    0| [노잼이네요.., 왜, 배우들이...| [노잼이네요, 왜, 배우들이, ...| [노잼이네요, 왜, 배우들이, ...|
|   제발 뻘짓 좀 그만하셨으면......|    0| [제발, 뻘짓, 좀, 그만하셨으...| [제발, 뻘짓, 좀, 그만하셨으...| [제발, 뻘짓, 좀, 그만하셨으...|
|         스토리가 매우 매우 아쉽다|    0| [스토리가, 매우, 매우, 아쉽다]| [스토리가, 매우, 매우, 아쉽다]| [스토리가, 매우, 매우, 아쉽다]|
|             영화 약간 이해가 안됨|    0|     [영화, 약간, 이해가, 안됨]|     [영화, 약간, 이해가, 안됨]|     [영화, 약간, 이해가, 안됨]|
|좋아하는 배우들인데 최근 영화라...|    0|[좋아하는, 배우들인데, 최근,...|[좋아하는, 배우들인데, 최근,...|[좋아하는, 배우들인데, 최근,...|
| 그지

## (3-3) TF-IDF를 계산

In [443]:
from pyspark.ml.feature import HashingTF, IDF
hashTF = HashingTF(inputCol="nostops", outputCol="features")
hashDf = hashTF.transform(review)

In [444]:
idf = IDF(inputCol="features", outputCol="idf")

idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

## 문제 3-3의 답

In [445]:
idfDf.select("nostops", "features","idf").show(3)

+------------------------------+--------------------+--------------------+
|                       nostops|            features|                 idf|
+------------------------------+--------------------+--------------------+
|[고구마, 먹은, 영화, 유치해...|(262144,[114067,1...|(262144,[114067,1...|
|[노잼이네요, 왜, 배우들이, ...|(262144,[14123,77...|(262144,[14123,77...|
|[제발, 뻘짓, 좀, 그만하셨으...|(262144,[24759,45...|(262144,[24759,45...|
+------------------------------+--------------------+--------------------+
only showing top 3 rows



## (3-4) 베이지안 모델을 적용해서 분류 (alpha는 1.0으로 설정)


In [446]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(featuresCol='features', labelCol='label', modelType='multinomial', predictionCol='prediction',smoothing=1.0)

In [447]:
model=nb.fit(hashDf)

In [448]:
prediction=model.transform(hashDf)

## 문제 3-4의 답

In [449]:
prediction.select('label','sent','prediction').show(47)

+-----+----------------------------------+----------+
|label|                              sent|prediction|
+-----+----------------------------------+----------+
|    0|고구마 먹은 영화 유치해서 못보겠다|       0.0|
|    0|  노잼이네요.. 왜 배우들이 출연...|       0.0|
|    0|   제발 뻘짓 좀 그만하셨으면......|       0.0|
|    0|         스토리가 매우 매우 아쉽다|       0.0|
|    0|             영화 약간 이해가 안됨|       0.0|
|    0|좋아하는 배우들인데 최근 영화라...|       0.0|
|    0| 그지같네 도대체 이 따위 영화를...|       0.0|
|    0|  스크린보다 시계를 더 많이 봄....|       0.0|
|    0| 이때까지 본 영화중에 제일 최악...|       0.0|
|    0|    나만 짜증.. 줄거리 한줄평.....|       0.0|
|    0|로맨틱 코미디로 방향을 잡았으면...|       0.0|
|    0| 똥 싸다가 막힌 느낌 안보는거를...|       0.0|
|    0|  어우 핵노잼 영화가 왜 이렇게 ...|       0.0|
|    0| 솔직히 재미없음 감독이 뭘 말할...|       0.0|
|    0|   뭘 말하고싶은지 모르겠다 아쉽다|       0.0|
|    0| 새벽에 케이블에서 볼거 없어서 ...|       0.0|
|    0|         핵노잼 억지 착한척 오졌다|       0.0|
|    0|오그라드는 대사는 기본에 옵션으...|       0.0|
|    0| 역대급 호화캐스팅인데 스토리, ...|       0.0|
|    0|  스토리 개 말도 안되는 억지 고...|       0.0|
|    0|

## (3-5) 훈련, 테스트 데이터를 반반씩 분리하고, 훈련 후 정확성을 계산하고 출력.

In [461]:
(trainDf, testDf) = hashDf.randomSplit([0.5, 0.5])

In [462]:
model_train=nb.fit(trainDf)

In [463]:
prediction=model_train.transform(testDf)

## 문제 3-5의 답

In [464]:
prediction.select('label','sent','prediction').show(10)

+-----+----------------------------------+----------+
|label|                              sent|prediction|
+-----+----------------------------------+----------+
|    0|고구마 먹은 영화 유치해서 못보겠다|       0.0|
|    0| 그냥 보지마세요 쓰레기 영화임 ...|       0.0|
|    0| 그지같네 도대체 이 따위 영화를...|       1.0|
|    1| 다음 날 새벽 출근임에도 불구하...|       1.0|
|    1| 단언컨대 한국 최고의 영화 복수...|       1.0|
|    0| 똥 싸다가 막힌 느낌 안보는거를...|       1.0|
|    0|   뭘 말하고싶은지 모르겠다 아쉽다|       0.0|
|    0|  생각보다 더 감흥이 없어요. 액...|       0.0|
|    0|  스크린보다 시계를 더 많이 봄....|       1.0|
|    0|  스토리 개 말도 안되는 억지 고...|       1.0|
+-----+----------------------------------+----------+
only showing top 10 rows



In [465]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator=BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label")
evaluator.evaluate(prediction)

0.7666666666666666

## (3-6) 앞서 훈련한 베이지안 모델로 다음 문장을 긍정, 부정으로 분류하세요

- 고구가 먹은 느낌 킬링타음으로도 아쉽다

- 내돈 아깝 핵노잼 영화가 이렇게 지루해서 못보겠다

- 보다가 채널 돌림 재미없다

- 최고의 영화

- 여운이 긴 명작

- 띵작입니다 몰입하고 숨도 못쉬고 봤어요 꼭 보세요

In [252]:
review_to_test=spark.createDataFrame(
    [
     ["고구가 먹은 느낌 킬링타음으로도 아쉽다"],
     ["내돈 아깝 핵노잼 영화가 이렇게 지루해서 못보겠다"],
     ["보다가 채널 돌림 재미없다"],
     ["최고의 영화"],
     ["여운이 긴 명작"],
     ["띵작입니다 몰입하고 숨도 못쉬고 봤어요 꼭 보세요"]
    ],['sent'])

In [253]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sent", outputCol="tokens")
rtt_token = tokenizer.transform(review_to_test)

In [254]:
rtt_wordsDf = rtt_token.withColumn('words', trimUdf(f.col('tokens')))
rtt_hashingTF = HashingTF(inputCol="nostops", outputCol="features")

In [255]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[stop,rtt_hashingTF])

In [256]:
rtt_md =pipeline.fit(rtt_wordsDf)
rtt_hashDf = rtt_md.transform(rtt_wordsDf)

In [257]:
rtt_hashDf.show()

+----------------------------------+-------------------------------+-------------------------------+-------------------------------+--------------------+
|                              sent|                         tokens|                          words|                        nostops|            features|
+----------------------------------+-------------------------------+-------------------------------+-------------------------------+--------------------+
|고구가 먹은 느낌 킬링타음으로도...| [고구가, 먹은, 느낌, 킬링타...| [고구가, 먹은, 느낌, 킬링타...| [고구가, 먹은, 느낌, 킬링타...|(262144,[8121,745...|
| 내돈 아깝 핵노잼 영화가 이렇게...| [내돈, 아깝, 핵노잼, 영화가...| [내돈, 아깝, 핵노잼, 영화가...| [내돈, 아깝, 핵노잼, 영화가...|(262144,[35595,47...|
|         보다가 채널 돌림 재미없다| [보다가, 채널, 돌림, 재미없다]| [보다가, 채널, 돌림, 재미없다]| [보다가, 채널, 돌림, 재미없다]|(262144,[103639,2...|
|                       최고의 영화|                 [최고의, 영화]|                 [최고의, 영화]|                 [최고의, 영화]|(262144,[17393,14...|
|                    여운이 긴 명작|             [여운이, 긴, 명작]|             [여운이

In [258]:
rtt_prediction=model_train.transform(rtt_hashDf)

## 문제 3-6의 답

In [259]:
rtt_prediction.select('sent','prediction').show()

+----------------------------------+----------+
|                              sent|prediction|
+----------------------------------+----------+
|고구가 먹은 느낌 킬링타음으로도...|       0.0|
| 내돈 아깝 핵노잼 영화가 이렇게...|       0.0|
|         보다가 채널 돌림 재미없다|       0.0|
|                       최고의 영화|       1.0|
|                    여운이 긴 명작|       1.0|
|띵작입니다 몰입하고 숨도 못쉬고...|       1.0|
+----------------------------------+----------+

